In [ ]:

%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_decoding_tools import glm_decoding_llr

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import raster_plot, raster_plot_3d

# import decoding
from neural_data_analysis.neural_analysis_tools.decoding_tools import decoding_utils, decoding_analysis, plot_decoding, cmp_decode, load_results
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_neural_viz import stop_event_raster

from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data

from neural_data_analysis.neural_analysis_tools.lfads import lfads_visualizer

from neural_data_analysis.neural_analysis_tools.neural_axes import window_search

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)

# retrieve data

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0326"

pn, datasets, comparisons = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path)

globals().update(datasets)

captures_df, valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    distance_col="distance_from_ff_to_stop",
)

keys = ['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle', 'one_stop_vs_both_first_miss', 'switch_vs_retry_after_miss', 'switch_vs_retry_after_retry']

cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.05,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

# call class

In [ ]:
# UPDATED IMPORT (NEW MODULAR STRUCTURE)
from neural_data_analysis.neural_analysis_tools.neural_axes.continuous_behavior_axis_analyzer import ContinuousBehaviorAxisAnalyzer


In [ ]:
window_a_ms = (0, 100)
window_b_ms = window_a_ms

# 1. Prepare comparison info
key = "switch_vs_retry_after_retry"
comp, A1, B1, a_label, b_label = get_stops_utils.extract_comparison_info(
    comparisons, datasets, key
)

# 2. Create analyzer
an = ContinuousBehaviorAxisAnalyzer(
    spikes_df=pn.spikes_df,
    event_a_df=A1,
    event_b_df=B1,
    event_col="stop_id_start_time",
    event_a_label=a_label,
    event_b_label=b_label,
    bin_width_ms=10,
    smoothing_sigma_ms=30,
)

# 3. Compute axis
axis_info = an.compute_event_axis(
    window_a_ms=window_a_ms,
    window_b_ms=window_b_ms,
    model="logreg"
)


cv = an.cross_validate_axis(window_a_ms, window_b_ms)
an.diagnose_axis(axis_info, cv_results=cv)


an.plot_event_aligned_projection(
    axis_info,
    window_ms=(-200, 400),
    ci="bootstrap",
    n_boot=500,
)


# more plots

In [ ]:
an.plot_projection_with_events(axis_info)


# Iterate

In [ ]:
stop!

In [ ]:
keys = [a['key'] for a in comparisons]

window_a_ms = (0, 100)
window_b_ms = window_a_ms

for key in keys:
    
    print('==============================================')
    print(f'{key}')

    comp, A1, B1, a_label, b_label = get_stops_utils.extract_comparison_info(comparisons, datasets, key)

    # 2. Create analyzer
    an = ContinuousBehaviorAxisAnalyzer(
        spikes_df=pn.spikes_df,
        event_a_df=A1,
        event_b_df=B1,
        event_col="stop_id_start_time",
        event_a_label=a_label,
        event_b_label=b_label,
        bin_width_ms=10,
        smoothing_sigma_ms=30,
    )

    # # 3. Compute axis
    # axis_info = an.compute_event_axis(
    #     window_a_ms=window_a_ms,
    #     window_b_ms=window_b_ms,
    #     model="logreg"
    # )

    # an.plot_event_aligned_projection(
    #     axis_info,
    #     window_ms=(-200, 400),
    #     ci="bootstrap",
    #     n_boot=500,
    #     title=f"{a_label} vs {b_label}"
    # )

    # cv = an.cross_validate_axis(window_a_ms, window_b_ms)
    # an.diagnose_axis(axis_info, cv_results=cv)
    
    # or directly find the best window
    dashboard = window_search.window_search_dashboard(
        analyzer=an,
        start_ms_values=np.arange(-200, 301, 50),
        end_ms_values=np.arange(0, 401, 50),
        model="logreg",
        n_splits=5,
        metric="mean_auc",
        heatmap_metric="mean_auc",
        window_ms_for_plot=(-200, 400),
        ci="bootstrap",
        n_boot=500,
        verbose=False
    )



# LFADS

In [ ]:
stop!

In [ ]:

from neural_data_analysis.neural_analysis_tools.lfads import lfads_integration
from neural_data_analysis.neural_analysis_tools.neural_axes.continuous_behavior_axis_analyzer import ContinuousBehaviorAxisAnalyzer


In [ ]:
lfads_out.keys()

In [ ]:
lfads_out['lfads_factors_mat'][1000:1050, :10]

In [ ]:
lfads_out['lfads_fr_mat'][:10, :10]

In [ ]:
save_dir = os.path.join(pn.processed_neural_data_folder_path, "lfads")
os.makedirs(save_dir, exist_ok=True)  # ensure it exists
out_path = os.path.join(save_dir, "lfads_bw10.pt")  # short name

# 1) Run LFADS on the session
lfads_out = lfads_integration.run_lfads_on_continuous_session(
    spikes_df=pn.spikes_df,
    device_str='cuda',
    out_path=out_path
)


lfads_fr_mat = lfads_out['lfads_fr_mat']
clusters = lfads_out['clusters']
start_time = lfads_out['start_time']

# 2) Build analyzer using LFADS FR instead of raw FR
key = 'switch_vs_retry_after_retry'
comp, A1, B1, a_label, b_label = get_stops_utils.extract_comparison_info(comparisons, datasets, key)

an_lfads = ContinuousBehaviorAxisAnalyzer(
    spikes_df=pn.spikes_df,
    event_a_df=A1,
    event_b_df=B1,
    event_col='stop_id_start_time',
    event_a_label=a_label,
    event_b_label=b_label,
    bin_width_ms=10,
    smoothing_sigma_ms=0.0,              # not used if external_fr_mat provided
    external_fr_mat=lfads_fr_mat,
    external_start_time=start_time,
    external_clusters=clusters,
)

axis_info = an_lfads.compute_event_axis(
    window_a_ms=(0, 100),
    window_b_ms=(0, 100),
    model='logreg'
)

cv_results = an_lfads.cross_validate_axis(
    window_a_ms=(0, 100),
    window_b_ms=(0, 100),
    model='logreg',
    n_splits=5,
)

print('LFADS-based mean CV AUC:', cv_results['mean_auc'])
print('LFADS-based cosine similarity:', cv_results['axis_cosine_similarity'])

an_lfads.plot_event_aligned_projection(
    axis_info,
    window_ms=(-200, 400),
    ci='bootstrap',
    n_boot=500,
    show_significance=True,
    alpha=0.05
)


In [ ]:
axis_info

# visualize lfads

In [ ]:
from neural_data_analysis.neural_analysis_tools.lfads import lfads_utils

# plot latent factors

## visualizer class

In [ ]:
from neural_data_analysis.neural_analysis_tools.lfads import lfads_visualizer

In [ ]:
out_path = os.path.join(pn.processed_neural_data_folder_path, save_dir, "lfads_bw10.pt") 

# 1) Run LFADS on the session
lfads_out = lfads_integration.run_lfads_on_continuous_session(
    spikes_df=pn.spikes_df,
    out_path=out_path,
    load_if_exists=True,
    overwrite=True
)


In [ ]:
lfads_out.keys()

In [ ]:
lfads_out['lfads_factors_mat'][1000:1100, 3]

In [ ]:

lfads_fr_mat = lfads_out['lfads_fr_mat']
lfads_factors_mat = lfads_out.get('lfads_factors_mat', None)
clusters = lfads_out['clusters']
start_time = lfads_out['start_time']

viz = lfads_visualizer.LFADSResultsVisualizer(
    spikes_df=pn.spikes_df,
    lfads_fr_mat=lfads_fr_mat,
    lfads_factors_mat=lfads_factors_mat,
    clusters=clusters,
    start_time=start_time,
    bin_width_ms=10.0,
)

# 1) Raw vs LFADS
viz.plot_raw_vs_lfads(neuron_index=0)

# 2) Factors
viz.plot_lfads_factors(n_factors=5)

# 3) Event-aligned FR (retry)
viz.plot_event_aligned(an.event_a_df, event_col='stop_id_start_time', neuron_index=0)

# 4) Event-aligned FR (switch)
viz.plot_event_aligned(an.event_b_df, event_col='stop_id_start_time', neuron_index=0)

# 5) PCA on factors
viz.plot_pca_factors()


# =====================================================================
# END OF FILE
# =====================================================================


## psth

In [ ]:
an = ContinuousBehaviorAxisAnalyzer(
    spikes_df=pn.spikes_df,
    event_a_df=A1,
    event_b_df=B1,
    event_col='stop_id_start_time',
    external_fr_mat=lfads_fr_mat,
    external_start_time=start_time,
    external_clusters=clusters,
)

psth_a, psth_b = an.compute_psth(window_ms=(-200, 600))


In [ ]:
an.plot_event_psth(...)


In [ ]:
from sklearn.decomposition import PCA

p = PCA(n_components=3).fit_transform(lfads_fr_mat)
plt.plot(p[:,0], label='PC1')
plt.plot(p[:,1], label='PC2')
plt.plot(p[:,2], label='PC3')
plt.legend()
plt.show()


# Window search

In [ ]:


dashboard = window_search.window_search_dashboard(
    analyzer=an,
    start_ms_values=np.arange(-200, 301, 50),
    end_ms_values=np.arange(0, 401, 50),
    model="logreg",
    n_splits=5,
    metric="mean_auc",
    heatmap_metric="mean_auc",
    window_ms_for_plot=(-200, 400),
    ci="bootstrap",
    n_boot=500
)


# multiple axes

In [ ]:
axes_specs = []

key = "switch_vs_retry_after_miss"
comp, A1, B1, a_label, b_label = get_stops_utils.extract_comparison_info(comparisons, datasets, key)
axes_specs.append({
    "label": key,
    "event_a_df": A1,     # switch
    "event_b_df": B1,      # retry
    "window_a_ms": (0, 150),
    "window_b_ms": (0, 150),
})


key = "switch_vs_retry_after_retry"
comp, A1, B1, a_label, b_label = get_stops_utils.extract_comparison_info(comparisons, datasets, key)
axes_specs.append({
    "label": key,
    "event_a_df": A1,     # switch
    "event_b_df": B1,      # retry
    "window_a_ms": (0, 150),
    "window_b_ms": (0, 150),
})


key = "captures_vs_all_misses"
comp, A1, B1, a_label, b_label = get_stops_utils.extract_comparison_info(comparisons, datasets, key)
axes_specs.append({
    "label": key,
    "event_a_df": A1,     # switch
    "event_b_df": B1,      # retry
    "window_a_ms": (0, 150),
    "window_b_ms": (0, 150),
})

In [ ]:
multi_axes = an.compute_multiple_axes(axes_specs)

In [ ]:
# Angle between any two axes

angle_retry_reward = an.axis_angle(
    multi_axes["switch_vs_retry_after_miss"]["axis_vec"],
    multi_axes["switch_vs_retry_after_retry"]["axis_vec"],
    #     multi_axes["captures_vs_all_misses"]["axis_vec"],
)
print("Angle (retry, reward):", angle_retry_reward)


In [ ]:
proj3 = an.project_3d(multi_axes["switch_vs_retry_after_miss"],
                      multi_axes["switch_vs_retry_after_retry"],
                      multi_axes["captures_vs_all_misses"])

an.plot_3d_projection(proj3)
